<a href="https://colab.research.google.com/github/Krish7Hack/Side-Channel-Attack-Resistant-Cryptographic-Libraries/blob/main/Side_Channel_Attack_Resistant_Cryptographic_Libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Side-Channel Attack Resistant Cryptographic Libraries
# =====================================================
# This notebook implements constant-time versions of AES, RSA, and ECC,
# and evaluates their resistance to timing side-channel attacks.


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import secrets
import random
from concurrent.futures import ThreadPoolExecutor

# Helper functions for timing measurements

In [ ]:
def measure_execution_time(func,*args,**kwargs):
  "Measure execution time of a function in nanoseconds"
  start = time.perf_counter_ns() #Time spent during sleep or system clock cahnges
  result = func(*args,**kwargs)
  end = time.perf_counter_ns() #Records the time after the functions finishes executing
  return result,end-start

In [ ]:
def collect_timing_samples(func,samples=1000,*args,**kwargs):
  "Collect timing sample for each function"
  times = []
  for _ in tqdm(range(samples)):
    _, execution_time = measure_execution_time(func,*args,**kwargs)
    times.append(execution_time)
  return times
#Here tqdm shows a smart progress bar for loop and iterable operations

In [ ]:
def plot_timing_comparison(timing_data,labels,title):
  "Plot timing comparison between implementation"
  plt.figure(figsize=(12,6))

  for i,times in enumerate(timing_data):
    plt.hist(times,bins=50,alpha=0.5,label=labels[i])

    plt.xlabel('Execution Time (ns)')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()
    plt.grid(True,alpha=0.3)
    plt.show()

    #Print Statistics
    for i ,times in enumerate(timing_data):
      print(f"{labels[i]}")
      print(f" Mean: {np.mean(times):.2f} ns")
      print(f"  Std Dev: {np.std(times):.2f} ns")
      print(f"  Variance: {np.var(times):.2f}")
      print(f"  Min: {np.min(times):.2f} ns")
      print(f"  Max: {np.max(times):.2f} ns")
      print()

# 1. Constant-Time AES Implementation
